# Data Wrangling
This data contains personality scores for recruits, plus the department they applied for at Umuzi.
Below I'll manipulate it with various functions to make it easy to view.

I begin by importing the relevant modules that we'll use.

In [1]:
import pandas as pd
import re
import numpy as np

Read the csv files.

In [2]:
#df_dep= pd.read_csv('data/departments.csv', sep=';', index_col=0)
df_dep= pd.read_csv('data/departments.csv', sep=';')
df_person= pd.read_csv('data/personality_scores.csv', sep=';')
df_dep.head()

,ID,Department
0,0,Data
1,1,Data
2,2,Data
3,3,Data
4,4,Data


drop any duplicate IDs.

In [3]:
df_person=df_person.drop_duplicates(subset='ID')


Make sure that te leng of both dataframes is equal. It will give an assertion error if not.

In [4]:
assert len(df_person)== len(df_dep)

create a function called scores which will be responsible for summing up the values in the given columns.

In [5]:

def scores(row, trait):
    """
    Takes in a row of a dataframe as well as a trait in the form of a digit.
    """
    total=0
    reg= re.compile('\d+')
    for col in row:
        numbers= reg.findall(col)
        if str(trait)==numbers[0]:
            total+= int(numbers[1])
            
    return total

#dictionary to link each number to a respective personal trait.
traits= dict({
    1 : 'Extraversion', 
    2 : 'Agreeableness',
    3 : 'Conscientiousness', 
    4 : 'Emotional Stability',
    5 : 'Intellect'})

#loop over each trait and add a column. calls the function 'scores' to be applied for each of those columns.
for each in traits.keys():
    df_person[f'{traits[each]}']= df_person.loc[:,'Section 5 of 6 [I am always prepared.]':'Section 5 of 6 [I worry about things.]'].apply(scores, args=(each,) , axis=1)

Names of departments are case sensitive, thus Web dev and web dev appear as two different categories. To solve this I made all the names lower case.

In [6]:
df_dep.Department.unique()

array(['Data', 'Web Dev', 'Copywriting', 'Design', 'Strategy', 'Web dev'],
      dtype=object)

In [7]:
df_dep['Department']=df_dep.Department.apply(lambda x: x.lower())

### merge the dataframes column-wise.

In [8]:
df_all= pd.merge(df_dep, df_person, on='ID').set_index('ID')
df_all.head()

,Department,Section 5 of 6 [I am always prepared.],Section 5 of 6 [I am easily disturbed.],Section 5 of 6 [I am exacting (demanding) in my work.],Section 5 of 6 [I am full of ideas.],Section 5 of 6 [I am interested in people.],Section 5 of 6 [I am not interested in abstract ideas.],Section 5 of 6 [I am not interested in other people's problems.],Section 5 of 6 [I am not really interested in others.],Section 5 of 6 [I am quick to understand things.],...,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,IPIP_HIGH_RISK,Extraversion,Agreeableness,Conscientiousness,Emotional Stability,Intellect
ID,,,,,,,,,,,,,,,,,,,,,
0,data,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 3)","(5, 3)","(2, 3)","(2, 5)","(5, 5)",...,NaN,NaN,NaN,NaN,NaN,30,40,48,36,42
1,data,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 5)","(5, 3)","(2, 5)","(2, 5)","(5, 5)",...,NaN,NaN,NaN,NaN,NaN,42,46,46,40,42
2,data,"(3, 5)","(4, 3)","(3, 3)","(5, 5)","(2, 5)","(5, 5)","(2, 5)","(2, 5)","(5, 5)",...,NaN,NaN,NaN,NaN,NaN,28,40,40,38,42
3,data,"(3, 5)","(4, 5)","(3, 3)","(5, 5)","(2, 5)","(5, 3)","(2, 3)","(2, 3)","(5, 3)",...,NaN,NaN,NaN,NaN,NaN,30,38,38,40,38
4,data,"(3, 3)","(4, 5)","(3, 3)","(5, 3)","(2, 3)","(5, 3)","(2, 3)","(2, 3)","(5, 5)",...,NaN,NaN,NaN,NaN,NaN,28,34,46,38,36


verify that indeed the length of the new dataframe is equal to any of the old ones.

In [9]:
assert len(df_all)== len(df_person)

### risk categorization

In [10]:
def risk(row):
    if row['Agreeableness']<30 and row['Conscientiousness']<30 and row['Emotional Stability']<30:
        return 'high risk'
    else:
        return 'low risk'
        

df_all['risk']= df_all[['Agreeableness', 'Conscientiousness', 'Emotional Stability']].apply(risk, axis=1)


In [11]:
df_high_risk= df_all[df_all.risk=='high risk']
df_high_risk[['Department', 'risk']]

,Department,risk
ID,,
881,data,high risk
1197,copywriting,high risk


## High and low risk within each department. To be fixed.

In [26]:
#df_all.groupby(['risk', 'Department']).aggregate('count').unstack().iloc[:,:5].fillna(0)
df_all.groupby(['risk', 'Department'])[['Agreeableness']].aggregate('count')

Agreeableness
risk      Department                
high risk copywriting              1
          data                     1
low risk  copywriting            325
          data                   328
          design                 120
          strategy               449
          web dev                331

It is evident from above that the total number of high risk applicants are 5, with none found in design and web dev. Astonishingly the remaining 1550 are considered to be low risk

In [13]:
df_all.head()

,Department,Section 5 of 6 [I am always prepared.],Section 5 of 6 [I am easily disturbed.],Section 5 of 6 [I am exacting (demanding) in my work.],Section 5 of 6 [I am full of ideas.],Section 5 of 6 [I am interested in people.],Section 5 of 6 [I am not interested in abstract ideas.],Section 5 of 6 [I am not interested in other people's problems.],Section 5 of 6 [I am not really interested in others.],Section 5 of 6 [I am quick to understand things.],...,Unnamed: 66,Unnamed: 67,Unnamed: 68,IPIP_HIGH_RISK,Extraversion,Agreeableness,Conscientiousness,Emotional Stability,Intellect,risk
ID,,,,,,,,,,,,,,,,,,,,,
0,data,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 3)","(5, 3)","(2, 3)","(2, 5)","(5, 5)",...,NaN,NaN,NaN,NaN,30,40,48,36,42,low risk
1,data,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 5)","(5, 3)","(2, 5)","(2, 5)","(5, 5)",...,NaN,NaN,NaN,NaN,42,46,46,40,42,low risk
2,data,"(3, 5)","(4, 3)","(3, 3)","(5, 5)","(2, 5)","(5, 5)","(2, 5)","(2, 5)","(5, 5)",...,NaN,NaN,NaN,NaN,28,40,40,38,42,low risk
3,data,"(3, 5)","(4, 5)","(3, 3)","(5, 5)","(2, 5)","(5, 3)","(2, 3)","(2, 3)","(5, 3)",...,NaN,NaN,NaN,NaN,30,38,38,40,38,low risk
4,data,"(3, 3)","(4, 5)","(3, 3)","(5, 3)","(2, 3)","(5, 3)","(2, 3)","(2, 3)","(5, 5)",...,NaN,NaN,NaN,NaN,28,34,46,38,36,low risk
